In [22]:
%history -g -f filename

In [1]:
from nltk import *

In [2]:
from nltk.corpus import *

In [3]:
import nltk

In [4]:
def language_ratio(input):
    lang_ratio={}
    tokens=wordpunct_tokenize(input)
    words=[word.lower()for word in tokens]
    for language in stopwords.fileids():
        stopwords_set=set(stopwords.words(language))
        words_set=set(words)
        common_elements=words_set.intersection(stopwords_set)
        lang_ratio[language]=len(common_elements)
    return lang_ratio
def detect_lang(input):
    ratios=language_ratio(input)
    language=max(ratios,key=ratios.get)
    return language

In [5]:
stopwords.fileids()

['arabic',
 'azerbaijani',
 'bengali',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hungarian',
 'indonesian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'slovene',
 'spanish',
 'swedish',
 'tajik',
 'turkish']

In [6]:
#nltk.download('stopwords')

In [7]:
messages = [line.rstrip() for line in open('Txt/104193_HELIS_2021_archivage.txt',encoding="utf8")]
print(len(messages))

8215


In [8]:
for message_no, message in enumerate(messages[:10]):
    print(message_no, message)
    print('\n')

0 Searching for neutrinoless double-beta decay with


1 scintillating bolometers


2 Dounia Lila Helis


3 


4 To cite this version:


5 Dounia Lila Helis. Searching for neutrinoless double-beta decay with scintillating bolometers. High


6 Energy Physics - Experiment [hep-ex]. Université Paris-Saclay, 2021. English. �NNT : 2021UPASP073�. �tel-03442659�


7 


8 HAL Id: tel-03442659


9 https://tel.archives-ouvertes.fr/tel-03442659




In [9]:
import pandas as pd

In [10]:
messages = pd.read_csv('Txt/104193_HELIS_2021_archivage.txt', sep='\t',
                           names=["Message", "Label"])
messages.head()

,Message,Label
0,Searching for neutrinoless double-beta decay with,NaN
1,scintillating bolometers,NaN
2,Dounia Lila Helis,NaN
3,To cite this version:,NaN
4,Dounia Lila Helis. Searching for neutrinoless ...,NaN


In [11]:
language=detect_lang(messages['Message'].iloc[9])
language

'english'

In [12]:
import nltk
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
pd.set_option("max_rows", 600)
from pathlib import Path  
import glob

In [14]:
directory_path = "../Txt"

In [15]:
text_files = glob.glob(f"{directory_path}/*.txt")

In [16]:
text_files

[]

In [17]:
text_titles = [Path(text).stem for text in text_files]

In [18]:
text_titles

[]

In [19]:
tfidf_vectorizer = TfidfVectorizer(input='filename', stop_words='english')

In [20]:
tfidf_vector = tfidf_vectorizer.fit_transform(text_files)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=text_titles, columns=tfidf_vectorizer.get_feature_names())

In [ ]:
tfidf_df.loc['00_Document Frequency'] = (tfidf_df > 0).sum()

In [ ]:
tfidf_df = tfidf_df.drop('00_Document Frequency', errors='ignore')

In [ ]:
tfidf_df.stack().reset_index()

In [ ]:
tfidf_df = tfidf_df.stack().reset_index()

In [ ]:
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})

In [ ]:
tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)

In [ ]:
top_tfidf = tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)

In [ ]:
!pip install altair

In [ ]:
import altair as alt
import numpy as np

# Terms in this list will get a red dot in the visualization
term_list = ['war', 'peace']

# adding a little randomness to break ties in term ranking
top_tfidf_plusRand = top_tfidf.copy()
top_tfidf_plusRand['tfidf'] = top_tfidf_plusRand['tfidf'] + np.random.rand(top_tfidf.shape[0])*0.0001

# base for all visualizations, with rank calculation
base = alt.Chart(top_tfidf_plusRand).encode(
    x = 'rank:O',
    y = 'document:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["document"],
)

# heatmap specification
heatmap = base.mark_rect().encode(
    color = 'tfidf:Q'
)

# red circle over terms in above list
circle = base.mark_circle(size=100).encode(
    color = alt.condition(
        alt.FieldOneOfPredicate(field='term', oneOf=term_list),
        alt.value('red'),
        alt.value('#FFFFFF00')        
    )
)

# text labels, white for darker heatmap colors
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)

# display the three superimposed visualizations
(heatmap + circle + text).properties(width = 600)